In [ ]:
import subprocess
import os
import numpy as np
import soundfile as sf

directory = './TIMITTRAIN/'
temp = './temp/'
out = "./TIMIT2/"

overlap = 0.30

new_files = [] #append filenames together with their list 

filenames=[]
for filename in os.listdir(directory):
    filenames.append(filename)

for f in range(2000):
    
    #random ids, draw random i and j until id1 and id2 are no longer equal
    id1 = "a"
    id2 = "a"
    
    while(id1 == id2):
        i, j, k = np.random.choice(len(filenames),3)
        id1 = filenames[i][0:8]
        id2 = filenames[j][0:8]
    
    input1 = directory+filenames[i]
    input2 = directory+filenames[j]
    
    in1f = sf.SoundFile(input1)
    length1 = round((len(in1f) / in1f.samplerate),1)
    
    in2f = sf.SoundFile(input2)
    length2 = round((len(in2f) / in2f.samplerate),1)
    
    
    in1 = 0
    in2 = 0
    len1 = 0
    len2 = 0
    if(length1<length2):
        in1 = input2
        in2 = input1
        len1 = length2
        len2 = length1
    else: 
        in1 = input1
        in2 = input2
        len1 = length1
        len2 = length2
        
    overlap_time = round((len1+len2)*overlap,1) #overlap_time = (len1+len2)*0.1
    start_time = round(len1-overlap_time,1) #start_time = len1+overlap_time
    
    start_time_rounded = int(start_time)
    
    
    totaltime = round(start_time+len2, 1) 
    if(not (totaltime % 1 == 0)):
        totaltime += 1
        
    totaltime = int(totaltime)
    
    lijstje = [0] * totaltime       
    for t in range(totaltime):
        if(t < start_time_rounded):
            lijstje[t] = 1
        if(t >= start_time_rounded and t <= len1):
            lijstje[t] = 2
        if(t >= start_time_rounded and t > len1):
            lijstje[t] = 1
    
    new_filename = "2_" + str(filenames[i][0:8]) + "_" + str(filenames[j][0:8]) + ".WAV"
    new_filename_lijstje = "2_" + str(filenames[i][0:8]) + "_" + str(filenames[j][0:8]) + "_s00"
    new_filename_trim = "2_" + str(filenames[i][0:8]) + "_" + str(filenames[j][0:8])  + "_s.WAV"
    temp_file = temp + new_filename 
    
    for l in range(len(lijstje)):
        new_files.append([new_filename_lijstje+str(l+1)+".WAV", lijstje[l]])
    
    padcommand = "sox " + str(in2) + " padded/" + str(filenames[j]) + " pad " + str(start_time) +" 0 "
    subprocess.run(padcommand)
    
    runcommand = "sox -m " + str(in1) + " padded/" + str(filenames[j]) +" "+ str(temp_file)
    subprocess.run(runcommand)
    
    normcommand = "sox " + str(temp_file) + " normalized/" + str(new_filename) + " norm "
    subprocess.run(normcommand)
    
    out_file = out + new_filename_trim
    
    trimcommand = "sox normalized/" + str(new_filename) + " " + str(out_file) + " trim 0 1 : newfile : restart "
    subprocess.run(trimcommand)
    

In [ ]:
import pandas as pd
df = pd.DataFrame(new_files, columns = ['Filename', 'Speakers'])
df.head()

In [ ]:
df.to_csv('2speakers_10000_30.csv', index=False)